# **Average Point Spread Function (PSF) distillation**

#### This notebook makes an average PSF out of a z-stack with acquired fluorescence beads. It will also perform statistical analysis over individual beads and averaged PSF. Accepted formats are ***czi*** and ***tif***.

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
from skimage.io import imread, imsave
from pyclesperanto_prototype import imshow
import pyclesperanto_prototype as cle
import pandas as pd
import matplotlib.pyplot as plt
from aicsimageio import AICSImage
import os
import tifffile
import stackview
import xml.etree.ElementTree as ET
from psf_functions import *

The main analysis is done using [pyclesperanto-prototype ](https://pypi.org/project/pyclesperanto-prototype/)

In [ ]:
from ipyfilechooser import FileChooser
fc = FileChooser()
display(fc)

In [ ]:
file_path = fc.selected_path + '\\' + fc.selected_filename
print("File path: ", file_path)
#path = os.path.abspath("")
#print(path)

In [ ]:
head, tail = os.path.split(file_path)

results_path = head + '\\results'  
if not os.path.exists(results_path):
    os.makedirs(results_path)

if tail.endswith('tif') or tail.endswith('czi'):
    file_name = tail[:-4]
else:
    print('Format not supported')

path_save = results_path + '\\' + file_name
print(path_save)

In [ ]:
if file_path[-3:] == 'czi':
    # Get an AICSImage object
    img = AICSImage(file_path)
    data = img.data
    root = img.metadata
    bead_image = cle.asarray(data[0][0])
    print(bead_image.shape)
    pixels, metadata_ome = metadata_read(file_path)
    print('Voxel size (x,y,z) in nm: ', pixels)
if file_path[-3:] == 'tif':
    bead_image = imread(file_path)
    print(bead_image.shape)
    pixels, metadata_ome = metadata_read(file_path)
    print('Voxel size (x,y,z) in nm: ', pixels)

In [ ]:
stackview.imshow(cle.maximum_x_projection(bead_image), colorbar=True)
stackview.imshow(cle.maximum_y_projection(bead_image), colorbar=True)
stackview.imshow(cle.maximum_z_projection(bead_image), colorbar=True)

In [ ]:
stackview.slice(bead_image, continuous_update=True, zoom_factor = 0.5)

For determining an average PSF, technically we can crop out all the individual beads, align them and then average the images. Therefore, we segment the objects and determine their center of mass.

In [ ]:
# Segment objects
label_image = cle.voronoi_otsu_labeling(bead_image)
stackview.imshow(label_image, labels=True)

In [ ]:
# determine center of mass for each object
stats = cle.statistics_of_labelled_pixels(bead_image, label_image)

df = pd.DataFrame(stats)
df[["mass_center_x", "mass_center_y", "mass_center_z"]]

# PSF averaging

Next, we will iterate over the beads and crop them out by translating them into a smaller PSF image. We calculate FWHM for each bead at the same time.

In [ ]:
# configure size of future PSF image
psf_radius = 20
size = psf_radius * 2 + 1

# initialize PSF
single_psf_image = cle.create([size, size, size])
avg_psf_image = cle.create([size, size, size])

# initialize FWHM
FWHM_x = []
FWHM_y = []
FWHM_zx = []
FWHM_zy = []

num_psfs = len(df)
indx = []
for index, row in df.iterrows():
    x = row["mass_center_x"]
    y = row["mass_center_y"]
    z = row["mass_center_z"]
    
    print("Bead", index, "at position", x, y, z)
    
    # move PSF in right position in a smaller image
    cle.translate(bead_image, single_psf_image, 
                  translate_x= -x + psf_radius,
                  translate_y= -y + psf_radius,
                  translate_z= -z + psf_radius)

    [fwhm_x, fwhm_y, fwhm_zx, fwhm_zy] = FWHM(size, pixels, single_psf_image)
    print('fwhm x: ', fwhm_x, 'nm', ' fwhm y: ', fwhm_y, 'nm', ' fwhm zx: ', fwhm_zx, 'nm', ' fwhm zy: ', fwhm_zy, 'nm')
    
    # visualize
    fig, axs = plt.subplots(1,3)
    axs[0].set_title("Bead " + str(index))
    #plt.title("Bead " + str(index))
    stackview.imshow(cle.maximum_x_projection(single_psf_image), plot=axs[0])
    stackview.imshow(cle.maximum_y_projection(single_psf_image), plot=axs[1])
    stackview.imshow(cle.maximum_z_projection(single_psf_image), plot=axs[2])

    #indx.append(index)
        # average
    avg_psf_image = avg_psf_image + single_psf_image / num_psfs
    
    FWHM_x = np.append(FWHM_x,fwhm_x)
    FWHM_y = np.append(FWHM_y,fwhm_y)
    FWHM_zx = np.append(FWHM_zx,fwhm_zx) 
    FWHM_zy = np.append(FWHM_zy,fwhm_zy)

In [ ]:
# mean and standard deviation of FWHM value
FWHM_x_mean = round(np.mean(FWHM_x))
FWHM_x_std = round(np.std(FWHM_x))
FWHM_y_mean =  round(np.mean(FWHM_y))
FWHM_y_std = round(np.std(FWHM_y))
FWHM_zx_mean = round(np.mean(FWHM_zx))
FWHM_zx_std = round(np.std(FWHM_zx))
FWHM_zy_mean = round(np.mean(FWHM_zy))
FWHM_zy_std = round(np.std(FWHM_zy))

print('FWHM in x: ', FWHM_x_mean, '+-',FWHM_x_std, 'nm')
print('FWHM in y: ', FWHM_y_mean, '+-',FWHM_y_std, 'nm')
print('FWHM in zx: ', FWHM_zx_mean, '+-',FWHM_zx_std, 'nm')
print('FWHM in zy: ', FWHM_zy_mean, '+-',FWHM_zy_std, 'nm')

In [ ]:
fig, axs = plt.subplots(nrows=1,ncols=3,subplot_kw={'xticks': [], 'yticks': []})
stackview.imshow(cle.maximum_x_projection(avg_psf_image), plot=axs[0])
stackview.imshow(cle.maximum_y_projection(avg_psf_image), plot=axs[1])
stackview.imshow(cle.maximum_z_projection(avg_psf_image), plot=axs[2])

In [ ]:
avg_psf_image.min(), avg_psf_image.max()

In [ ]:
normalized_psf = avg_psf_image / np.sum(avg_psf_image)

stackview.imshow(normalized_psf, colorbar=True)

In [ ]:
normalized_psf.min(), normalized_psf.max()

**Saving projection tif image of averaged PSF**

In [ ]:
fig.savefig(path_save + "_projections.tif")

**Saving average PSF stack in ome.tif file**

In [ ]:
# Save the numpy array as OME TIFF with metadata
tifffile.imwrite(path_save + '_avgPSF_stack.ome.tiff', normalized_psf, metadata=metadata_ome)
#imsave(path_save + '_avgPSF_stack.tif', normalized_psf)

### Analysis of PSF

Data [conversion](https://github.com/clEsperanto/pyclesperanto_prototype/blob/master/demo/interoperability/numpy.ipynb) from pyclesperanto to numpy. 

In [ ]:
z_projection = np.asarray(cle.maximum_z_projection(avg_psf_image))
x_projection = np.asarray(cle.maximum_x_projection(avg_psf_image))
y_projection = np.asarray(cle.maximum_y_projection(avg_psf_image))

In [ ]:
# Indexes of the maximum in image array:
ind_x = np.unravel_index(np.argmax(x_projection, axis=None), x_projection.shape)
ind_y = np.unravel_index(np.argmax(y_projection, axis=None), y_projection.shape)
ind_z = np.unravel_index(np.argmax(z_projection, axis=None), z_projection.shape)

print('x: ', ind_x)
print('y: ', ind_y)
print('z: ', ind_z)

#### FWHM for z-axis

In [ ]:
units_xy, units_z, units_xynew, units_znew = units(size, pixels)

In [ ]:
# Normalized curve
zx_resolution = x_projection[ind_x[0],:]/avg_psf_image.max()
zy_resolution = y_projection[:,ind_y[1]]/avg_psf_image.max()

Cubic Spline Interpolation from [Python Numerical Methods](https://pythonnumericalmethods.studentorg.berkeley.edu/notebooks/chapter17.03-Cubic-Spline-Interpolation.html) for z-axis in x and y planes.

In [ ]:
f_zx = CubicSpline(units_z,zx_resolution, bc_type='natural')
f_zy = CubicSpline(units_z,zy_resolution, bc_type='natural')
units_znew = np.linspace(units_z[0], units_z[-1], 1000)
fzx_new = f_zx(units_znew)
fzy_new = f_zy(units_znew)

indices_zx = np.where(fzx_new > 0.5)[0].tolist()
fwhm_zx = round(units_znew[indices_zx[-1]]-units_znew[indices_zx[0]])
indices_zy = np.where(fzy_new > 0.5)[0].tolist()
fwhm_zy = round(units_znew[indices_zy[-1]]-units_znew[indices_zy[0]])

[matplotlib](https://matplotlib.org/stable/users/explain/quick_start.html)

In [ ]:
fig, ax = plt.subplots(figsize=(5, 5))
ax.plot(units_znew,fzx_new,'black')
ax.plot(units_znew, fzy_new, 'black')
ax.plot(units_z,zx_resolution, 'ro', label='zx')
ax.plot(units_z,zy_resolution, 'bo', label='zy')
ax.set_title('FWHM')
ax.legend()
plt.show()
print('zx fwhm: ', fwhm_zx, 'nm')
print('zy fwhm: ', fwhm_zy, 'nm')

In [ ]:
fig.savefig(path_save + "_z_fwhm.svg")

#### FWHM for xy plane

In [ ]:
x_resolution = z_projection[ind_z[0],:]/avg_psf_image.max()
y_resolution = z_projection[:,ind_z[1]]/avg_psf_image.max()

Cubic Spline Interpolation from [Python Numerical Methods](https://pythonnumericalmethods.studentorg.berkeley.edu/notebooks/chapter17.03-Cubic-Spline-Interpolation.html) for xy (lateral) resolution (in x and y direction)

In [ ]:
f_x = CubicSpline(units_xy,x_resolution, bc_type='natural')
f_y = CubicSpline(units_xy,y_resolution, bc_type='natural')
units_xynew = np.linspace(units_xy[0], units_xy[-1], 1000)
fx_new = f_x(units_xynew)
fy_new = f_y(units_xynew)

indices_x = np.where(fx_new > 0.5)[0].tolist()
fwhm_x = round(units_xynew[indices_x[-1]]-units_xynew[indices_x[0]])
indices_y = np.where(fy_new > 0.5)[0].tolist()
fwhm_y = round(units_xynew[indices_y[-1]]-units_xynew[indices_y[0]])

[matplotlib](https://matplotlib.org/stable/users/explain/quick_start.html)

In [ ]:
fig, ax = plt.subplots(figsize=(5, 5))
ax.plot(units_xynew,fx_new,'black')
ax.plot(units_xy,x_resolution, 'ro', label='x')
ax.set_title('FWHM')
ax.legend()
plt.show()
print('x fwhm: ', fwhm_x, 'nm') 

In [ ]:
fig.savefig(path_save + "_x_fwhm.svg")

In [ ]:
fig, ax = plt.subplots(figsize=(5, 5))
ax.plot(units_xynew, fy_new, 'black')
ax.plot(units_xy,y_resolution, 'bo', label='y')
ax.set_title('FWHM')
ax.legend()
plt.show()
print('y fwhm: ', fwhm_y, 'nm') 

In [ ]:
fig.savefig(path_save + "_y_fwhm.svg")

#### Lateral and axial resolution

Choose which direction you want to use: 
x vs. y in lateral
zx vs. zy in axial

In [ ]:
lateral = x_resolution
axial = zy_resolution

In [ ]:
if np.all(lateral == x_resolution):
    lateral_fit = fx_new
    lateral_fwhm = fwhm_x
    which_xy = '(x)'
else:
    lateral_fit = fy_new
    lateral_fwhm = fwhm_y
    which_xy = '(y)'
if np.all(axial == zx_resolution):
    axial_fit = fzx_new
    axial_fwhm = fwhm_zx
    which_z = '(zx)'
else: 
    axial_fit = fzy_new
    axial_fwhm = fwhm_zy
    which_z = '(zy)'

In [ ]:
fig, ax = plt.subplots(figsize=(5, 5))
ax.plot(units_xynew, lateral_fit,'black')
ax.plot(units_xy, lateral, 'ro', label='lateral')
ax.plot(units_znew, axial_fit, 'black')
ax.plot(units_z, axial, 'bo', label='axial')
ax.set_title('FWHM')
ax.set_xlabel('Distance (nm)')
ax.set_ylabel('Normalized intensity (a.u.)')
ax.legend()
plt.show()
print('lateral ' + which_xy + ': ', lateral_fwhm, 'nm')
print('axial ' + which_z + ': ', axial_fwhm, 'nm')

Values for lateral and axial resolution come from the averaged psf calculated above and these values has no error by default. 
We can use the FWHM values from each bead and calculate standard deviation (calculated above as well) for the mean FWHM value with an error. Nevertheless, those two value slightly differs.

**Saving the graph in the file**

In [ ]:
fig.savefig(path_save + "_fwhm.svg")

**Saving the data in the csv file**

In [ ]:
axis = ["lateral", "axial"]
datatype = ["data", "Cubic Spline"]
datapoints = {"distance_lateral": units_xy, "lateral": x_resolution,
              "distance_axial": units_z, "axial": zy_resolution}            
spline = {"distance_lateral": units_xynew, "CS_lateral": fx_new,
                 "distance_axial": units_znew, "CS_axial": fzy_new}

df_data = pd.DataFrame(datapoints)
df_spline = pd.DataFrame(spline)

df_data.to_csv(path_save + '_datapoints.csv')
df_spline.to_csv(path_save + '_spline.csv')

In [ ]:
with open(path_save + '_fwhm.txt', 'w') as f:
    f.write('FWHMs of ' + '%d' %len(df_trunc) + ' beads:' + '\n' +
            'FWHM in x: ' + '%d' %FWHM_x_mean + ' +- ' + '%d' %FWHM_x_std + ' nm' + '\n' +
            'FWHM in y: ' + '%d' %FWHM_y_mean + ' +- ' + '%d' %FWHM_y_std + ' nm' + '\n' +    
            'FWHM in zx: ' + '%d' %FWHM_zx_mean + ' +- ' + '%d' %FWHM_zx_std + ' nm' + '\n' + 
            'FWHM in zy: ' + '%d' %FWHM_zy_mean + ' +- ' + '%d' %FWHM_zy_std + ' nm' + '\n' + '\n' +
            'Averaged PSF out of ' + '%d' %len(df_trunc) + ' beads:' +  '\n' + 
            'Lateral ' + which_xy + ': ' + '%d' %lateral_fwhm + ' nm' + '\n' +
            'Axial ' + which_z + ': ' + '%d' %axial_fwhm + ' nm' + '\n') 

**Saving individual projection in tif files**

In [ ]:
fig, ax = plt.subplots()
plt.imshow(cle.maximum_z_projection(normalized_psf),cmap='gray')
plt.axis('off')
plt.show()
fig.savefig(path_save + "_z_projection.tif")

In [ ]:
fig, ax = plt.subplots()
plt.imshow(cle.maximum_y_projection(normalized_psf),cmap='gray')
plt.axis('off')
plt.show()
fig.savefig(path_save + "_y_projection.tif")

In [ ]:
fig, ax = plt.subplots()
plt.imshow(cle.maximum_x_projection(normalized_psf),cmap='gray')
plt.axis('off')
plt.show()
fig.savefig(path_save + "_x_projection.tif")